In [1]:
import random
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from datasets import load_dataset



d:\HocTap\ChatBot\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load a model
from dotenv import load_dotenv
import os
load_dotenv()
hf_token = os.environ["HF_TOKEN"]
model = SentenceTransformer("dangvantuan/vietnamese-document-embedding", trust_remote_code=True, token=hf_token)

In [3]:
# Load the Touche-2020 IR dataset (https://huggingface.co/datasets/BeIR/webis-touche2020, https://huggingface.co/datasets/BeIR/webis-touche2020-qrels)
corpus = load_dataset("another-symato/VMTEB-Zalo-legel-retrieval-wseg", "corpus", split="train", token=hf_token)
queries = load_dataset("another-symato/VMTEB-Zalo-legel-retrieval-wseg", "queries", split="train", token=hf_token)
relevant_docs_data = load_dataset("another-symato/VMTEB-Zalo-legel-retrieval-wseg", "data_ir", split="train", token=hf_token)

In [4]:
# For this dataset, we want to concatenate the title and texts for the corpus
corpus = corpus.map(lambda x: {'text': x['title'] + " " + x['text']}, remove_columns=['title'])

In [5]:
# Shrink the corpus size heavily to only the relevant documents + 5,000 random documents
required_corpus_ids = set(map(str, relevant_docs_data["corpus_id"]))
required_corpus_ids |= set(random.sample(corpus["id"], k=5_000))
corpus = corpus.filter(lambda x: x["id"] in required_corpus_ids)

Filter: 100%|██████████| 61425/61425 [00:00<00:00, 139444.03 examples/s]


In [6]:
# Convert the datasets to dictionaries
corpus = dict(zip(corpus["id"], corpus["text"]))  # Our corpus (cid => document)
queries = dict(zip(queries["query_id"], queries["question"]))  # Our queries (qid => question)
relevant_docs = {}  # Query ID to relevant documents (qid => set([relevant_cids])
for qid, corpus_ids in zip(relevant_docs_data["query_id"], relevant_docs_data["corpus_id"]):
    qid = str(qid)
    corpus_ids = str(corpus_ids)
    if qid not in relevant_docs:
        relevant_docs[qid] = set()
    relevant_docs[qid].add(corpus_ids)

In [7]:
# Given queries, a corpus and a mapping with relevant documents, the InformationRetrievalEvaluator computes different IR metrics.
ir_evaluator = InformationRetrievalEvaluator(
    queries=queries,
    corpus=corpus,
    relevant_docs=relevant_docs,
    name="dangvantuan/vietnamese-document-embedding",
)

In [8]:
corpus_ids = list(corpus.keys())
corpus_texts = list(corpus.values())

# Encode trước
corpus_embeddings = model.encode(
    corpus_texts,
    batch_size=4,  # giảm batch size
    convert_to_tensor=True,
    device="cuda"  # hoặc "cpu"
)

In [9]:
# Đánh giá với embedding đã có
results = ir_evaluator(model, corpus_embeddings=corpus_embeddings)

In [10]:
print(ir_evaluator.primary_metric)
# => "BeIR-touche2020-test_cosine_map@100"
print(results[ir_evaluator.primary_metric])
# => 0.29335196224364596

dangvantuan/vietnamese-document-embedding_cosine_ndcg@10
0.6172178745231993


In [11]:
from pprint import pprint
pprint(results)

{'dangvantuan/vietnamese-document-embedding_cosine_accuracy@1': 0.4624413145539906,
 'dangvantuan/vietnamese-document-embedding_cosine_accuracy@10': 0.7777777777777778,
 'dangvantuan/vietnamese-document-embedding_cosine_accuracy@3': 0.6396713615023474,
 'dangvantuan/vietnamese-document-embedding_cosine_accuracy@5': 0.7046165884194053,
 'dangvantuan/vietnamese-document-embedding_cosine_map@100': 0.5726857517200948,
 'dangvantuan/vietnamese-document-embedding_cosine_mrr@10': 0.5661990461286232,
 'dangvantuan/vietnamese-document-embedding_cosine_ndcg@10': 0.6172178745231993,
 'dangvantuan/vietnamese-document-embedding_cosine_precision@1': 0.4624413145539906,
 'dangvantuan/vietnamese-document-embedding_cosine_precision@10': 0.07777777777777778,
 'dangvantuan/vietnamese-document-embedding_cosine_precision@3': 0.21322378716744914,
 'dangvantuan/vietnamese-document-embedding_cosine_precision@5': 0.14092331768388106,
 'dangvantuan/vietnamese-document-embedding_cosine_recall@1': 0.4624413145539